In [43]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [44]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama, OllamaEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpointEmbeddings

llm = ChatOllama(model="gemma3:4b")
# embeddings = HuggingFaceEndpointEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")


In [45]:
data = PyPDFLoader("D:\Projects2025\Chatbot2\Media\operating system.pdf").load()

spliter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=200)

docs =  spliter.split_documents(data)
docs

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\97naf\AppData\Local\Temp\ipykernel_21320\4092517892.py:1: SyntaxWarning: invalid escape sequence '\P'
  data = PyPDFLoader("D:\Projects2025\Chatbot2\Media\operating system.pdf").load()


[Document(metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 0, 'page_label': '1'}, page_content='Operating System Questions & Answers – Basics\n--------------------------------------------------------------------------------\n---------------------------------\n1. What is an operating system?\na) collection of programs that manages hardware resources\nb) system service provider to the application programs\nc) interface between the hardware and application programs\nd) all of the mentioned\n Answer: d\nExplanation: An Operating System acts as an intermediary between user/user \napplications/application programs and hardware. It is a program that manages \nhardware resources. It provides services to application programs.\n2. To access the services of operating system, the interface is provided by the

In [46]:
from langchain_community.vectorstores import FAISS


vs = FAISS.from_documents(embedding=embeddings, documents=docs)

vs

In [47]:
vs.similarity_search("What is an operating system")

[Document(id='94e8def4-6646-41da-ac37-604851f6f776', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 102, 'page_label': '103'}, page_content='terminates itself in an orderly manner, it is called?\na) Asynchronous cancellation\nb) Systematic cancellation\nc) Sudden Termination\nd) Deferred cancellation\n Answer: d'),
 Document(id='9282155f-8fc6-499d-84b5-57af96d83c23', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 98, 'page_label': '99'}, page_content='b) the kernel cannot schedule another thread in the same application for \nexecution\nc) the kernel must schedule another thread of a different applicatio

In [48]:
retriver = vs.as_retriever()

In [49]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "you are an helpful assistant help to resolve this query based on the {context}. if the query is not relevent say i don't know "),
        ("human", "{input}")
    ]
)

In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


context_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)
rag = create_retrieval_chain(retriver, context_chain)
rag

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E30A28D810>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="you are an helpful assistant help to resolve this query based on the {context}. if the query is not relevent say i don't know "), additiona

In [51]:
response = rag.invoke({"input": "What is an hululu?"})
response

{'input': 'What is an hululu?',
 'context': [Document(id='94e8def4-6646-41da-ac37-604851f6f776', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 102, 'page_label': '103'}, page_content='terminates itself in an orderly manner, it is called?\na) Asynchronous cancellation\nb) Systematic cancellation\nc) Sudden Termination\nd) Deferred cancellation\n Answer: d'),
  Document(id='9282155f-8fc6-499d-84b5-57af96d83c23', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 98, 'page_label': '99'}, page_content='b) the kernel cannot schedule another thread in the same application for \nexecution\nc) the kernel must sche

In [52]:
response = rag.invoke({"input": "What is my name?"})
response

{'input': 'What is my name?',
 'context': [Document(id='94e8def4-6646-41da-ac37-604851f6f776', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 102, 'page_label': '103'}, page_content='terminates itself in an orderly manner, it is called?\na) Asynchronous cancellation\nb) Systematic cancellation\nc) Sudden Termination\nd) Deferred cancellation\n Answer: d'),
  Document(id='e53f12c6-cf0f-4c6c-9d2c-b7e4c71a54eb', metadata={'producer': 'LibreOffice 24.8.1.2 (X86_64) / LibreOffice Community', 'creator': 'Writer', 'creationdate': '2024-10-16T16:21:41+00:00', 'source': 'D:\\Projects2025\\Chatbot2\\Media\\operating system.pdf', 'total_pages': 179, 'page': 21, 'page_label': '22'}, page_content='5. What is FIFO algorithm?\na) first executes the job that came in last in the queue\nb) first executes the job

In [53]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("chat_history"),
    ]
)

In [54]:

h_ritriver = create_history_aware_retriever(llm, retriver, contextualize_q_prompt)
h_ritriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E30A28D810>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMess

In [55]:
stuff_doc = create_stuff_documents_chain(llm, prompt)
rag = create_retrieval_chain(h_ritriver, stuff_doc)

In [56]:
question = "What is an operating system?"
response1=rag.invoke({"input":question})
response1["answer"]

"Okay, let's break down what an operating system (OS) is.\n\n**Simply put, an operating system is the software that manages all of the hardware and software resources of a computer system.**  It's the intermediary between you (the user) and the hardware.\n\nHere's a more detailed explanation, broken down into key aspects:\n\n**1. Core Functions:**\n\n* **Resource Management:** This is the OS's primary job. It manages:\n    * **CPU Time:**  Decides which programs get to use the processor and for how long.\n    * **Memory:** Allocates and manages memory space for programs.\n    * **Storage:**  Manages files and directories on storage devices (hard drives, SSDs, etc.).\n    * **Input/Output (I/O) Devices:** Controls devices like keyboards, mice, printers, monitors, and network cards.\n* **Process Management:**  An OS creates, schedules, and terminates processes (running programs).\n* **File System Management:** Organizes files and directories on storage devices.\n* **User Interface:** Pro

In [57]:
question = "To access the services of it, the interface is provided by the?"
response1=rag.invoke({"input":question})
response1["answer"]

'Please provide the context! I need to know what "it" refers to in order to answer your question about the interface. 😊 \n\nFor example, are you asking about:\n\n*   A specific operating system?\n*   A software application?\n*   A hardware device?\n\nOnce you tell me what "it" is, I can give you a relevant answer.'

In [58]:
question = "Hello my name is Nafeesh?"
response1=rag.invoke({"input":question})
response1["answer"]

'Hello Nafeesh! It’s nice to meet you. 😊 \n\nIs there anything I can help you with today?'

In [59]:
question = "What is my name?"
response1=rag.invoke({"input":question})
response1["answer"]

'I don’t know your name. As an AI, I have no memory of past conversations or personal information about you. \n\nI’m here to help you with questions and tasks, but I don’t retain details from our interactions.'

In [60]:
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question= "Hello my name is Nafeesh?"
response1=rag.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="Tell me more about it?"
response2=rag.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

ResponseError: failed to create new sequence: no input provided (status code: 500)

In [ ]:
question2="What is my name?"
response2=rag.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

Hello Nafeesh! It’s nice to meet you. 😊 

Is there anything I can help you with today?
